In [137]:
from classiq import *
import numpy as np
import math

In [221]:
# Assuming these won't change
x0 = np.array([1, 1])
M = np.array([[0, 1], [-1, 0]])

# Free to change variables
t = 0.1
K = 7# Accuracy of solution - subject to change
T = math.ceil(math.log2(K+1))

In [222]:
def C(m):
   return 2 ** ((m + 1) / 2) * t ** m / math.factorial(m)

N = math.sqrt(sum([C(i) for i in range(K+1)])) # This also equals to $\mathcal{C}$ in the paper

state = []
print(2 ** T)
for i in range(2 ** T):
   if len(state) < K + 1:
      state.append(math.sqrt(C(i)) / N)

   else:
      state.append(0)

print(np.array(state) ** 2)

8
[8.68123445e-01 1.22771195e-01 8.68123445e-03 4.09237317e-04
 1.44687241e-05 4.09237317e-07 9.64581606e-09 1.94874913e-10]


In [223]:
def proj(v1, v2):
    """Calculates the projection of v1 onto v2"""
    return np.dot(v1, v2) / np.dot(v2, v2) * np.array(v2)

def gram_schmidt():
    """
    Performs Gram Schmidt procedure to create VS1 and WS1
    
    NOTE: The way it is currently implemented K+1 must be a multiple of 2 to
        guarantee obtaining a unitary matrix
    """
    WS1 = [state]
    
    for i in range(2 ** T):
        basis_v = [0 for _ in range(2 ** T)]
        basis_v[i] = 1
        s = np.array(basis_v[:])
    
        for v in WS1:
            s = s - proj(s, v)
    
        if np.linalg.norm(s) != 0:
            s = s / np.linalg.norm(s)
            WS1.append(s.tolist())
        
    WS1 = WS1[:2 ** T]
    VS1 = np.transpose(WS1).tolist()

    return (VS1, WS1)

In [224]:
# TODO Initialize ancilla registers with the correct quantum states
# NOTE Our b vector is zero, therefore C-V_S2 and C-U_b do not need to be implemented, but could be for future purposes


print("Register sizes:", 1, T, 1)
VS1, WS1 = gram_schmidt()

@qfunc
def encode_stage(ancilla_reg_1: QBit, ancilla_reg_2: QNum, work_reg: QBit):
    # Z(ancilla_reg_1)
    unitary(VS1, ancilla_reg_2)
    H(work_reg)


@qfunc
def create_entanglement(ancilla_reg_2: QNum, work_reg: QBit):
   repeat(
      count=K + 1,
      iteration=lambda i: if_(
         condition=(i % 2 != 0),
         then=lambda: control(
               ctrl=(ancilla_reg_2 == i), stmt_block=lambda: Y(work_reg)
            ),
         ),
      )

@qfunc
def main(ancilla_reg_1: Output[QBit], ancilla_reg_2: Output[QNum], work_reg: Output[QBit]):
   allocate(1, ancilla_reg_1)
   allocate(T, ancilla_reg_2)
   allocate(1, work_reg) # 2x2 matrix, will change for higher order ODEs if we get to it

   encode_stage(ancilla_reg_1, ancilla_reg_2, work_reg)
   create_entanglement(ancilla_reg_2, work_reg)
   unitary(WS1, ancilla_reg_2)
   # Z(ancilla_reg_1)

Register sizes: 1 3 1


In [225]:
# View circuit on Classiq IDE
qmod = create_model(main)
qprog = synthesize(qmod)
job = execute(qprog)
results = job.result()[0].value.parsed_counts
results

[{'ancilla_reg_1': 0.0, 'ancilla_reg_2': 0.0, 'work_reg': 0.0}: 844,
 {'ancilla_reg_1': 0.0, 'ancilla_reg_2': 0.0, 'work_reg': 1.0}: 770,
 {'ancilla_reg_1': 0.0, 'ancilla_reg_2': 1.0, 'work_reg': 0.0}: 210,
 {'ancilla_reg_1': 0.0, 'ancilla_reg_2': 1.0, 'work_reg': 1.0}: 193,
 {'ancilla_reg_1': 0.0, 'ancilla_reg_2': 2.0, 'work_reg': 1.0}: 15,
 {'ancilla_reg_1': 0.0, 'ancilla_reg_2': 2.0, 'work_reg': 0.0}: 11,
 {'ancilla_reg_1': 0.0, 'ancilla_reg_2': 3.0, 'work_reg': 0.0}: 2,
 {'ancilla_reg_1': 0.0, 'ancilla_reg_2': 3.0, 'work_reg': 1.0}: 2,
 {'ancilla_reg_1': 0.0, 'ancilla_reg_2': 4.0, 'work_reg': 0.0}: 1]

In [227]:
# Solution at time t
# Currently incorrect. Need to figure our what to do when solution is negative.
for r in results:
    if r.state['ancilla_reg_2'] == 0 and r.state['work_reg'] == 1:
        print(f"Solution at time t = {t} : {r.shots * N ** 2 / 2048}")

Solution at time t = 0.1 : 0.6124833474326662
